In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
## Import dataset from folder  ./data and create a dataframe
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


# Data cleaning + exploration

In [4]:
# Look for nan values in both train and test
print("train shape", train.shape)
print("test shape", test.shape)

print("train nan values", train.isna().sum())


train shape (2000, 21)
test shape (1000, 21)
train nan values battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64


No further cleaning is needed, as there are no nan values in the dataset.

In [21]:
# Concat test and train dataframe on column number
df = pd.concat([train, test], axis=0, sort=False)

# Drop coluimn id
df.drop('id', axis=1, inplace=True)

# Export to pandas dataframe file
df.to_csv('../data/dataset_full.csv', index=False)

In [9]:
df.shape

(3000, 21)

## Correlation matrix
With the correlation matrix we can see that the target variable is most correlated with the following features:

In [17]:
# Get correlations of columns in dataset
corr = df.corr()
# Plot correlations
# plt.figure(figsize=(20,20))
# sns.heatmap(corr, annot=True, cmap=plt.cm.Reds)
# plt.show()
# Calculate correlations with target variable
cor_target = abs(corr["price_range"])

# Select highly correlated features
relevant_features = cor_target[(cor_target>0.1) | (cor_target < -0.0)]
relevant_features


battery_power    0.200723
px_height        0.148858
px_width         0.165818
ram              0.917046
price_range      1.000000
Name: price_range, dtype: float64

In [19]:
# Apply pca to reduce dimensionality with variance 0.95
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
pca.fit(df.drop('price_range', axis=1))
pca.n_components_



4

In [20]:
# # Feature Importance: Random Forest
# from sklearn.ensemble import RandomForestClassifier
#
# # Create a random forest classifier
# clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
#
# # Train the classifier
# clf.fit(df.drop('price_range', axis=1), df['price_range'])
#
# # get redundant first features
# feature_importances = pd.DataFrame(clf.feature_importances_,
#                                    index = df.drop('price_range', axis=1).columns,
#                                     columns=['importance']).sort_values('importance', ascending=False)



ValueError: Input y contains NaN.

# Data splitting
Will be done in separate notebook with training

In [26]:
from scipy.stats import shapiro

# assuming your dataframe is called "df"
norm_cols = []
non_norm_cols = []

for column in df.columns:
    stat, p = shapiro(df[column])
    alpha = 0.05
    if p > alpha:
        norm_cols.append(column)
    else:
        non_norm_cols.append(column)

X_norm = df[norm_cols]
X_non_norm = df[non_norm_cols]

print("Normal distributed columns: ", norm_cols)
print("Non normal distributed columns: ", non_norm_cols)

Normal distributed columns:  ['price_range']
Non normal distributed columns:  ['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi']
